In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
"SET THE WORKING DIRECTORY BELOW TO THE LOCATION OF DATA FILES"

working_directory = 'C:/Users/conno/OneDrive/University Study/Honours Thesis/cnolan-thesis' #set location using back slashes

os.chdir(working_directory)

print("Current working directory: {0}".format(os.getcwd()))


def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
            output_path = os.makedirs(directory)
            print(output_path)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        

# Folder where outputs will be saved (by default a folder within the working directory) 
createFolder('./output/') 
output_path = working_directory +'./output/'

print('Set WD: Done')

Current working directory: C:\Users\conno\OneDrive\University Study\Honours Thesis\cnolan-thesis
Set WD: Done


# Import Data

In [3]:
nger_data_matched = pd.read_csv('https://raw.githubusercontent.com/connorpn/cnolan-thesis/main/output/nger_data_matched.csv',encoding = "ISO-8859-1")
ms_data = pd.read_csv ('https://raw.githubusercontent.com/connorpn/cnolan-thesis/main/output/ms_data.csv', encoding='latin1')
ds_industry = pd.read_csv ('https://raw.githubusercontent.com/connorpn/cnolan-thesis/main/data/ds_ticker_industry.csv')

# Combine Matched NGER, Morningstar, and Datasets 

In [4]:
emissions_determinants_data = pd.merge(ms_data[['ticker', 'morningstar_name', 'year', 'capex', 'eoy_price', 'eps', 'ltdebt', 'marketcap', 'ppe', 'revenue', 'roe', 'stdebt', 'assets', 'liabilities']],
                                  nger_data_matched[['year', 'ticker','matched', 'scope1', 'scope2', 'energy_consumption', 'total_emissions']],
                                  on = ['ticker','year'],
                                  how = 'left')

emissions_determinants_data = pd.merge(emissions_determinants_data,
                                       ds_industry,
                                      on=['ticker'],
                                      how='left')


emissions_determinants_data = emissions_determinants_data.reindex(columns=['year','ticker','morningstar_name','industry','matched', 'scope1', 'scope2', 'energy_consumption', 'total_emissions', 'capex', 'eoy_price', 'eps', 'ltdebt', 'marketcap', 'ppe', 'revenue', 'roe', 'stdebt', 'assets', 'liabilities'])



# Construct Dependent Variables (Carbon Emissions)

In [5]:
"Construct Log(emissions)"


#log scope1, scope2, total_emissions, and energy consumption
emissions_determinants_data['log_scope1'] = np.log(emissions_determinants_data['scope1'])
emissions_determinants_data['log_scope2'] = np.log(emissions_determinants_data['scope2'])
emissions_determinants_data['log_total_emissions'] = np.log(emissions_determinants_data['total_emissions'])
emissions_determinants_data['log_energy_consumption'] = np.log(emissions_determinants_data['energy_consumption'])


"Construct year by year change in emissions"

#get a list of unique company names
unique_firms_emissions_determinants_data = (emissions_determinants_data['ticker'].unique())

#clone each unique company for each observation year (2009-2021)
firms_2009 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2009'})
firms_2010 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2010'})
firms_2011 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2011'})
firms_2012 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2012'})
firms_2013 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2013'})
firms_2014 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2014'})
firms_2015 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2015'})
firms_2016 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2016'})
firms_2017 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2017'})
firms_2018 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2018'})
firms_2019 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2019'})
firms_2020 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2020'})
firms_2021 = pd.DataFrame({'ticker':unique_firms_emissions_determinants_data, 'year': '2021'})

#create dataframe to add all cloned firm years
firms_allyears = pd.DataFrame(columns = ['ticker', 'year'])

#add all clone firm year dataframes to a list
firm_years_list = [firms_2009, firms_2010, firms_2011, firms_2012, firms_2013, firms_2014, firms_2015, firms_2016, firms_2017, firms_2018, firms_2019, firms_2020, firms_2021]

#concatenate dataframes
firms_allyears = pd.concat(firm_years_list)

#convert year column to date time format
firms_allyears['year'] =  pd.to_datetime(firms_allyears['year'], format='%Y').dt.to_period("Y")
firms_allyears['year'] = firms_allyears['year'].astype(str)
#create dummy level index
emissions_determinants_data["dummy_index"] = emissions_determinants_data["dummy_index"] = 1
firms_allyears['dummy_index'] = firms_allyears['dummy_index'] = 2

#concenate cloned firm years with main data file
emissions_determinants_data = pd.concat([emissions_determinants_data, firms_allyears])

#sort dataframe by corporation name and year
emissions_determinants_data = emissions_determinants_data.sort_values(by=['ticker', 'year','dummy_index'])

#drop duplicates keeping first row (main  data file) as by dummy_index sorting
emissions_determinants_data = emissions_determinants_data.drop_duplicates(['ticker', 'year'], keep='first')

#sort dataframe by ticker and year
emissions_determinants_data = emissions_determinants_data.sort_values(by=['ticker', 'year'])

#calculate yearly change in emissions by corporation for scope1, scope2, total_emissions, and energy consumption
emissions_determinants_data['change_scope1'] = emissions_determinants_data.groupby(['ticker'])['scope1'].diff()
emissions_determinants_data['change_scope2'] = emissions_determinants_data.groupby(['ticker'])['scope2'].diff()
emissions_determinants_data['change_total_emissions'] = emissions_determinants_data.groupby(['ticker'])['total_emissions'].diff()
emissions_determinants_data['change_energy_consumption'] = emissions_determinants_data.groupby(['ticker'])['energy_consumption'].diff()


"Construct Intensity (Emissions)"

## int = (tons CO 2 e/AUD m.)
emissions_determinants_data['revenue(m)'] = emissions_determinants_data['revenue'] / 1000000
emissions_determinants_data['scope1_int'] = emissions_determinants_data['scope1'] /  emissions_determinants_data['revenue(m)']
emissions_determinants_data['scope2_int'] = emissions_determinants_data['scope2'] /  emissions_determinants_data['revenue(m)']
emissions_determinants_data['total_emissions_int'] = emissions_determinants_data['total_emissions'] /  emissions_determinants_data['revenue(m)']
emissions_determinants_data['energy_consumption_int'] = emissions_determinants_data['energy_consumption'] / emissions_determinants_data['revenue(m)']

C:\Users\conno\anaconda3\envs\spyder-env\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Construct Independent Variables

In [6]:
#LOGSIZE
emissions_determinants_data['logsize'] = np.log(emissions_determinants_data['marketcap'])

#B/M
emissions_determinants_data['bm'] = (emissions_determinants_data['assets'] - emissions_determinants_data['liabilities']) / emissions_determinants_data['marketcap']

#ROE (already exisits 'roe')

#LEVERAGE
emissions_determinants_data['totaldebt'] = emissions_determinants_data.stdebt.fillna(0) + emissions_determinants_data.ltdebt.fillna(0) #calculate total debt, skipping nan values (this means total debt can be constructed from ltdebt, stdebt, or both)
emissions_determinants_data['leverage'] = emissions_determinants_data.totaldebt / emissions_determinants_data.assets #calculate leverage

#INVEST/A
emissions_determinants_data['investa'] = emissions_determinants_data.capex / emissions_determinants_data.assets

#HHI
#HHI WILL NOT BE INCLUDED DUE TO ABSENCE OF SEGEMENTED BUSINESS REVENUES

#LOGPPE
emissions_determinants_data['logppe'] = np.log(emissions_determinants_data['ppe'])

#SALESGR
emissions_determinants_data = emissions_determinants_data.sort_values(by=['ticker', 'year']) #sort dataframe by ticker and year
emissions_determinants_data['revenue_change'] = emissions_determinants_data.groupby(['ticker'])['revenue'].diff() #calculate yearly change in revenue by firm
emissions_determinants_data['salesgr'] = emissions_determinants_data.revenue_change / emissions_determinants_data.marketcap #salesgr = change in annual revenue normailzed by marketcap

#EPSGR 
emissions_determinants_data = emissions_determinants_data.sort_values(by=['ticker', 'year']) #sort dataframe by ticker and year
emissions_determinants_data['eps_change'] = emissions_determinants_data.groupby(['ticker'])['eps'].diff() #calculate yearly change in eps by firm
emissions_determinants_data['epsgr'] = emissions_determinants_data.eps_change / emissions_determinants_data.eoy_price

C:\Users\conno\anaconda3\envs\spyder-env\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Filter and Process Dataset for Regression

In [7]:
emissions_determinants_data = emissions_determinants_data.dropna(axis=0, how= 'any', subset=['matched']).reset_index(drop=True)
emissions_determinants_data['year'] = emissions_determinants_data['year'].astype(int)
emissions_determinants_data = emissions_determinants_data.loc[(emissions_determinants_data['year'] >= 2009) & (emissions_determinants_data['year'] <= 2021)].reset_index(drop=True)
emissions_determinants_data = emissions_determinants_data.sort_values(by=['ticker', 'year']).reset_index(drop=True)

emissions_determinants_vars = emissions_determinants_data[['year', 'ticker','industry','log_scope1', 'log_scope2', 'log_total_emissions', 'log_energy_consumption', 'change_scope1', 'change_scope2', 'change_total_emissions', 'change_energy_consumption', 'scope1_int', 'scope2_int', 'total_emissions_int', 'energy_consumption_int', 'logsize', 'bm', 'roe','leverage', 'investa', 'logppe', 'salesgr', 'epsgr']]
emissions_determinants_vars = emissions_determinants_vars.sort_values(by=['year','ticker']).reset_index(drop=True)
emissions_determinants_vars = emissions_determinants_vars.dropna(how='all', subset=['log_scope1', 'log_scope2', 'log_total_emissions', 'log_energy_consumption', 'change_scope1', 'change_scope2', 'change_total_emissions', 'change_energy_consumption', 'scope1_int', 'scope2_int', 'total_emissions_int', 'energy_consumption_int']).reset_index(drop=True)
emissions_determinants_vars = emissions_determinants_vars.dropna(how='any', subset=['logsize', 'bm', 'roe', 'leverage', 'investa', 'logppe', 'salesgr', 'epsgr','industry']).reset_index(drop=True)

"Save Determinants of Carbon Emissions Variables Dataset (for regression use)"
output_filename = 'emissions_determinants_vars.csv'
outputname = output_path + output_filename
emissions_determinants_vars.to_csv(outputname, mode='w',index=False)
print("Exported File: " + outputname)

Exported File: C:/Users/conno/OneDrive/University Study/Honours Thesis/cnolan-thesis./output/emissions_determinants_vars.csv


# Constructed Variables for The Determinants of Carbon Emissions

In [8]:
print('Number of Observations:')
print (len(emissions_determinants_vars))

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(emissions_determinants_vars)

Number of Observations:
440


year ticker  industry  log_scope1  log_scope2  log_total_emissions  \
0    2009   ABY1      54.0   11.417944   10.721548            22.930875   
1    2009    AGL      91.0   14.300246   10.875837            25.813172   
2    2009    AIO      81.0   13.841222   11.351582            25.354148   
3    2009    AMP     104.0    8.995413   12.120155            22.810946   
4    2009    AOE      50.0   12.702907   10.580886            24.215834   
..    ...    ...       ...         ...         ...                  ...   
435  2021    ILU     122.0   12.762962   11.213522            24.275889   
436  2021    ING      35.0   10.999413   11.988880            24.814926   
437  2021    IPL      92.0   14.217526   11.981309            28.033036   
438  2021    JBH      90.0    5.886104   11.074219            17.400819   
439  2021    LAU     131.0   11.650955    9.218606            23.163882   

     log_energy_consumption  change_scope1  change_scope2  \
0                 14.421822            NaN            NaN   
1                 17.523290            NaN            NaN   
2                 16.526223            NaN            NaN   
3                 13.584176            NaN            NaN   
4                 15.220808            NaN            NaN   
..                      ...            ...            ...   
435               15.407556       -35367.0         1196.0   
436               14.359522         2232.0        -6205.0   
437               17.238024       -14512.0        10989.0   
438               12.586599          113.0        -2042.0   
439               14.332815         2409.0          888.0   

     change_total_emissions  change_energy_consumption  scope1_int  \
0                       NaN                        NaN  233.969419   
1                       NaN                        NaN  236.380656   
2                       NaN                        NaN  365.101850   
3                       NaN                        NaN    0.710536   
4                       NaN                        NaN  481.958555   
..                      ...                        ...         ...   
435           -3.536699e+09                  -346920.0  222.507172   
436            2.231994e+09                   -28980.0   22.422528   
437           -1.451199e+10                   979519.0  341.286301   
438            1.129796e+07                    -1117.0    0.040363   
439            1.035618e+10                    40457.0  260.198774   

     scope2_int  total_emissions_int  energy_consumption_int    logsize  \
0    116.605219         2.339706e+07             4717.659045  17.596790   
1      7.698568         2.363807e+07             5934.200052  22.518247   
2     30.281486         3.651022e+07             5351.739566  21.210094   
3     16.167548         7.105518e+05               69.897287  23.341766   
4     57.733504         4.819591e+07             5977.510693  21.670279   
..          ...                  ...                     ...        ...   
435   47.253140         2.225076e+07             3132.396889  22.175896   
436   60.312137         2.242259e+07              645.597107  21.114825   
437   36.470548         3.412863e+08             6996.883105  22.465510   
438    7.230295         4.043557e+03               32.807602  22.482984   
439   22.853322         2.601990e+07             3802.087465  18.538895   

           bm     roe  leverage   investa     logppe   salesgr       epsgr  
0    9.373302 -0.1849  0.228132 -0.085659  19.885266 -6.429864 -412.571429  
1    0.971150 -0.0854  0.123989 -0.065403  21.600525  0.208178   22.855762  
2    1.354374  0.0039  0.622072 -0.062456  21.704654 -0.072005   -3.120301  
3    0.192051  0.2952  0.137482  0.000000  19.978825  0.048705    0.886263  
4    0.450362 -0.0206  0.163082 -0.424470  20.280653  0.222553   13.822535  
..        ...     ...       ...       ...        ...       ...         ...  
435  0.373063  0.2506  0.013618 -0.020332  20.760754  0.130429  -47.946535  
436  0.110594

In [9]:
print('Notebook Finish')

Notebook Finish
